# LightGBM Install

In [1]:
# !git clone --recursive https://github.com/Microsoft/LightGBM
# %cd /content/LightGBM
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install --precompile

Cloning into 'LightGBM'...
remote: Enumerating objects: 26260, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 26260 (delta 25), reused 8 (delta 3), pack-reused 26197
Receiving objects: 100% (26260/26260), 19.06 MiB | 12.78 MiB/s, done.
Resolving deltas: 100% (19405/19405), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects: 100% (5/5), done.        
remote: Compressing

# Setup

In [2]:
import os
# python global seed
os.environ['PYTHONHASHSEED'] = str(42)
# tensorflow seed (not working for GPU)
# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
# weight and bias
os.environ["WANDB_API_KEY"] = "b838b62906ab267778c6e05b913ba6c4a27699b2"

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# # TextVectorization layer in tf 2.6 don't support "sparse=True" option which is used on TF-IDF
# # !pip install -q tensorflow==2.6.0
!pip install -q tensorflow==2.8
!pip install -q tensorflow-recommenders==0.6
# !pip install -q scann==1.2.3
# !pip install -q scann==1.2.6
# # !pip install -q tensorflow-datasets
!pip install -q tensorflow-addons
# !pip install -q tensorflow-hub
# # !pip install -q keras-tuner

# !pip install -q transformers

!pip install -q statsmodels
!pip install -q xgboost
# !pip install -q lightgbm
!pip install -q catboost
!pip install -q missingpy

!pip install -q optuna
!pip install -q wandb

Mounted at /content/drive
     / 668.3 MB 415 kB/s
     |████████████████████████████████| 462 kB 4.2 MB/s 
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 1.1 MB 4.3 MB/s 
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 49 kB 2.6 MB/s 
     |████████████████████████████████| 308 kB 4.3 MB/s 
     |████████████████████████████████| 81 kB 10.3 MB/s 
     |████████████████████████████████| 209 kB 88.1 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 146 kB 84.4 MB/s 
     |████████████████████████████████| 49 kB 6.1 MB/s 
     |████████████████████████████████| 112 kB 84.9 MB/s 
     |████████████████████████████████| 1.8 MB 4.3 MB/s 
     |████████████████████████████████| 181 kB 73.7 MB/s 
     |████████████████████████████████| 145 kB 74.3 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 


In [4]:
import sys
import shutil
from glob import glob
import multiprocessing as mp
import gc
from pathlib import Path
from scipy import stats
from scipy.special import boxcox, softmax
from scipy import sparse
import itertools

from multiprocessing import cpu_count
import copy
import pickle
import warnings
from datetime import datetime, timedelta
from time import time, sleep, mktime
from matplotlib import font_manager as fm, rc, rcParams
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
import random as rnd
import psutil
from optuna import Trial, create_study
from optuna.samplers import TPESampler
import wandb

import numpy as np
from numpy import array, nan, random as np_rnd, where
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from pandas import DataFrame as dataframe, Series as series, isna, read_csv
from pandas.tseries.offsets import DateOffset

from sklearn.model_selection import train_test_split as tts, StratifiedKFold, GroupKFold, GroupShuffleSplit, StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, KBinsDiscretizer
from sklearn import metrics
# from sklearn.compose import ColumnTransformer
# config_missingpy(); from missingpy import MissForest
# from sklearn.impute import KNNImputer
# from sklearn.feature_extraction.text import TfidfVectorizer

# try:
#     # RAPIDS config
#     os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
#     os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
#     os.environ['CONDA_PREFIX'] = '/usr/local'
#     import cudf as cd
#     import cupy as cp
#     from cuml.cluster import KMeans
#     from cuml.neighbors import NearestNeighbors
#     from cuml.metrics.cluster import silhouette_score
# except:
#     print("RAPIDS Import ERROR")

import xgboost as xgb
import lightgbm as lgb
import catboost as cat

# ===== tensorflow =====
import tensorflow as tf
from tensorflow import random as tf_rnd
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras import metrics as tf_metrics
from tensorflow.keras import callbacks as tf_callbacks
from tqdm.keras import TqdmCallback
import tensorflow_addons as tfa
from tensorflow.keras.utils import plot_model
from keras.utils.layer_utils import count_params

# import keras_tuner as kt
# from keras_tuner import HyperModel
# import tensorflow_hub as tf_hub
import tensorflow_recommenders as tfrs

# # # ===== pytorch =====
# import torch
# from torch.utils.data import DataLoader
# from transformers import AutoTokenizer
# from transformers import AutoModel

import librosa

# GPU memory setting
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)

warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)
# plt.rc('font', family='NanumSquareB')

In [5]:
# ===== utility functions =====
# label encoding for categorical column with excepting na value
def seed_everything(seed=42):
    # python random module
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # tf random
    try:
        tf_rnd.set_seed(seed)
    except:
        pass
    # RAPIDS random
    try:
        cp.random.seed(seed)
    except:
        pass
    # pytorch random
    try:
        torch.manual_seed(seed)
    except:
        pass
def which(bool_list):
    return where(bool_list)[0]
def easyIO(x=None, path=None, op="r"):
    tmp = None
    if op == "r":
        with open(path, "rb") as f:
            tmp = pickle.load(f)
        return tmp
    elif op == "w":
        with open(path, "wb") as f:
            pickle.dump(x, f)
    else:
        print("Unknown operation type")
def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]
def orderElems(for_order, using_ref):
    return [i for i in using_ref if i in for_order]
# concatenate by row
def cbr(df1, df2):
    if type(df1) == series:
        tmp_concat = series(pd.concat([dataframe(df1), dataframe(df2)], axis=0, ignore_index=True).iloc[:,0])
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == dataframe:
        tmp_concat = pd.concat([df1, df2], axis=0, ignore_index=True)
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == np.ndarray:
        tmp_concat = np.concatenate([df1, df2], axis=0)
    else:
        print("Unknown Type: return 1st argument")
        tmp_concat = df1
    return tmp_concat
def change_width(ax, new_value):
    for patch in ax.patches :
        current_width = patch.get_width()
        adj_value = current_width - new_value
        # we change the bar width
        patch.set_width(new_value)
        # we recenter the bar
        patch.set_x(patch.get_x() + adj_value * .5)
def week_of_month(date):
    month = date.month
    week = 0
    while date.month == month:
        week += 1
        date -= timedelta(days=7)
    return week
def getSeason(date):
    month = date.month
    if month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Fall"
    else:
        return "Winter"
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)
def sigmoid(x):
    return 1/(1 + np.exp(-x))
def dispPerformance(result_dic):
    perf_table = dataframe()
    index_names = []
    for k, v in result_dic.items():
        index_names.append(k)
        perf_table = pd.concat([perf_table, series(v["performance"]).to_frame().T], ignore_index=True, axis=0)
    perf_table.index = index_names
    perf_table.sort_values(perf_table.columns[0], inplace=True)
    print(perf_table)
    return perf_table
def powspace(start, stop, power, num):
    start = np.power(start, 1/float(power))
    stop = np.power(stop, 1/float(power))
    return np.power(np.linspace(start, stop, num=num), power)
def xgb_custom_lossfunction(alpha = 1):
    def support_under_mse(label, pred):
        # grad : 1차 미분
        # hess : 2차 미분
        residual = (label - pred).astype("float")
        grad = np.where(residual > 0, -2 * alpha * residual, -2 * residual)
        hess = np.where(residual > 0, 2 * alpha, 2.0)
        return grad, hess
    return support_under_mse
def pd_flatten(df):
    df = df.unstack()
    df.index = [str(i) + "_" + str(j) for i, j in df.index]
    return df
def tf_losses_rmse(y_true, y_pred, sample_weight=None):
    return tf.sqrt(tf.reduce_mean((y_true - y_pred) ** 2)) if sample_weight is None else tf.sqrt(tf.reduce_mean(((y_true - y_pred) ** 2) * sample_weight))
def tf_loss_nmae(y_true, y_pred, sample_weight=False):
    mae = tf.reduce_mean(tf.math.abs(y_true - y_pred))
    score = tf.math.divide(mae, tf.reduce_mean(tf.math.abs(y_true)))
    return score
def text_extractor(string, lang="eng", spacing=True):
    # # 괄호를 포함한 괄호 안 문자 제거 정규식
    # re.sub(r'\([^)]*\)', '', remove_text)
    # # <>를 포함한 <> 안 문자 제거 정규식
    # re.sub(r'\<[^)]*\>', '', remove_text)
    if lang == "eng":
        text_finder = re.compile('[^ A-Za-z]') if spacing else re.compile('[^A-Za-z]')
    elif lang == "kor":
        text_finder = re.compile('[^ ㄱ-ㅣ가-힣+]') if spacing else re.compile('[^ㄱ-ㅣ가-힣+]')
    # default : kor + eng
    else:
        text_finder = re.compile('[^ A-Za-zㄱ-ㅣ가-힣+]') if spacing else re.compile('[^A-Za-zㄱ-ㅣ가-힣+]')
    return text_finder.sub('', string)
def memory_usage(message='debug'):
    # current process RAM usage
    p = psutil.Process()
    rss = p.memory_info().rss / 2 ** 20 # Bytes to MB
    print(f"[{message}] memory usage: {rss: 10.3f} MB")
    return rss
def cos_sim(a, b):
  return dot(a, b)/(norm(a) * norm(b))
class MyLabelEncoder:
    def __init__(self, preset={}):
        # dic_cat format -> {"col_name": {"value": replace}}
        self.dic_cat = preset
    def fit_transform(self, data_x, col_names):
        tmp_x = copy.deepcopy(data_x)
        for i in col_names:
            # if key is not in dic, update dic
            if i not in self.dic_cat.keys():
                tmp_dic = dict.fromkeys(sorted(set(tmp_x[i]).difference([nan])))
                label_cnt = 0
                for j in tmp_dic.keys():
                    tmp_dic[j] = label_cnt
                    label_cnt += 1
                self.dic_cat[i] = tmp_dic
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def transform(self, data_x):
        tmp_x = copy.deepcopy(data_x)
        for i in self.dic_cat.keys():
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def clear(self):
        self.dic_cat = {}
class MyOneHotEncoder:
    def __init__(self, label_preset={}):
        self.dic_cat = {}
        self.label_preset = label_preset
    def fit_transform(self, data_x, col_names):
        tmp_x = dataframe()
        for i in data_x:
            if i not in col_names:
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                self.dic_cat[i] = OneHotEncoder(sparse=False, handle_unknown="ignore")
                conv = self.dic_cat[i].fit_transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j:
                                col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def transform(self, data_x):
        tmp_x = dataframe()
        for i in data_x:
            if not i in list(self.dic_cat.keys()):
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                conv = self.dic_cat[i].transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j: col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def clear(self):
        self.dic_cat = {}
        self.label_preset = {}
class MyKNNImputer:
    def __init__(self, k=5):
        self.imputer = KNNImputer(n_neighbors=k)
        self.dic_cat = {}
    def fit_transform(self, x, cat_vars=None):
        if cat_vars is None:
            x_imp = dataframe(self.imputer.fit_transform(x), columns=x.columns)
        else:
            naIdx = dict.fromkeys(cat_vars)
            for i in cat_vars:
                self.dic_cat[i] = diff(list(sorted(set(x[i]))), [nan])
                naIdx[i] = list(which(array(x[i].isna())))
            x_imp = dataframe(self.imputer.fit_transform(x), columns=x.columns)

            # if imputed categorical value are not in the range, adjust the value
            for i in cat_vars:
                x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
                for j in naIdx[i]:
                    if x_imp[i][j] not in self.dic_cat[i]:
                        if x_imp[i][j] < self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][0]
                        elif x_imp[i][j] > self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][len(self.dic_cat[i]) - 1]
        return x_imp
    def transform(self, x):
        if len(self.dic_cat.keys()) == 0:
            x_imp = dataframe(self.imputer.transform(x), columns=x.columns)
        else:
            naIdx = dict.fromkeys(self.dic_cat.keys())
            for i in self.dic_cat.keys():
                naIdx[i] = list(which(array(x[i].isna())))
            x_imp = dataframe(self.imputer.transform(x), columns=x.columns)

            # if imputed categorical value are not in the range, adjust the value
            for i in self.dic_cat.keys():
                x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
                for j in naIdx[i]:
                    if x_imp[i][j] not in self.dic_cat[i]:
                        if x_imp[i][j] < self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][0]
                        elif x_imp[i][j] > self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][len(self.dic_cat[i]) - 1]
        return x_imp
    def clear(self):
        self.imputer = None
        self.dic_cat = {}
def remove_outlier(df, std=3, mode="remove"):
    tmp_df = df.copy()
    if mode == "remove":
        outlier_mask = (np.abs(stats.zscore(tmp_df)) > std).all(axis=1)
        print("found outlier :", outlier_mask.sum())
        tmp_df = tmp_df[~outlier_mask]
    elif mode == "interpolate":
        tmp_outlier = []
        for i in tmp_df:
            outlier_mask = (np.abs(stats.zscore(tmp_df[i])) > std)
            tmp_outlier.append(outlier_mask.sum())
            if tmp_outlier[-1] == 0:
                continue
            tmp_df[i][outlier_mask] = np.nan
            tmp_df[i] = tmp_df[i].interpolate(method='linear').bfill()
        print("found outlier :", np.sum(outlier_mask))
    return tmp_df
def convert_sparse_matrix_to_sparse_tensor(X, sorted=True):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape)) if sorted else tf.SparseTensor(indices, coo.data, coo.shape)
seed_everything()

In [6]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/projects/Dacon/covid19_diagnostics/"

CFG = {
    'SR':16000,
    'N_MFCC':32, # MFCC 벡터를 추출할 개수
    'SEED':41
}

# Preprocessing

In [ ]:
df_full = pd.read_csv(folder_path + 'open.zip (Unzipped Files)/train_data.csv')
df_test = pd.read_csv(folder_path + 'open.zip (Unzipped Files)/test_data.csv')

In [ ]:
df_full.head()

,id,age,gender,respiratory_condition,fever_or_muscle_pain,covid19
0,1,24,female,0,1,0
1,2,51,male,0,0,0
2,3,22,male,0,0,0
3,4,29,female,1,0,0
4,5,23,male,0,0,0


In [ ]:
df_test.head()

,id,age,gender,respiratory_condition,fever_or_muscle_pain
0,3806,48,female,1,0
1,3807,24,female,0,0
2,3808,29,male,0,0
3,3809,39,female,0,0
4,3810,34,male,0,0


In [ ]:
def get_mfcc_feature(df, data_type):
    # Data Folder path
    root_folder = folder_path + 'open.zip (Unzipped Files)/'
    features = []
    for uid in tqdm(df['id']):
        root_path = os.path.join(root_folder, data_type)
        path = os.path.join(root_path, str(uid).zfill(5)+'.wav')

        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=CFG['SR'])
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)
    
    # 기존의 자가진단 정보를 담은 데이터프레임에 추출된 오디오 Feature를 추가
    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    df = pd.concat([df, mfcc_df], axis=1)
    return df
    print('Done.')

In [ ]:
df_full = get_mfcc_feature(df_full, 'train')
df_test = get_mfcc_feature(df_test, 'test')

100%|██████████| 5732/5732 [1:59:49<00:00,  1.25s/it]


In [ ]:
easyIO(df_full, folder_path + "dataset/train_mfcc.csv", "w")
easyIO(df_test, folder_path + "dataset/test_mfcc.csv", "w")

# Quick Start Session

In [42]:
target_var = "covid19"
num_vars = ["age"]
bin_vars = ["gender", "respiratory_condition", "fever_or_muscle_pain"]
cat_vars = []

mfcc_vars = ["mfcc_" + str(i) for i in range(1, CFG["N_MFCC"]+1)]

In [43]:
df_full = easyIO(None, folder_path + "dataset/train_mfcc.csv", "r")
df_test = easyIO(None, folder_path + "dataset/test_mfcc.csv", "r")

In [44]:
df_full.drop("id", axis=1, inplace=True)
df_test.drop("id", axis=1, inplace=True)

In [45]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3805 entries, 0 to 3804
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   age                    3805 non-null   int64  
 1   gender                 3805 non-null   object 
 2   respiratory_condition  3805 non-null   int64  
 3   fever_or_muscle_pain   3805 non-null   int64  
 4   covid19                3805 non-null   int64  
 5   mfcc_1                 3805 non-null   float32
 6   mfcc_2                 3805 non-null   float32
 7   mfcc_3                 3805 non-null   float32
 8   mfcc_4                 3805 non-null   float32
 9   mfcc_5                 3805 non-null   float32
 10  mfcc_6                 3805 non-null   float32
 11  mfcc_7                 3805 non-null   float32
 12  mfcc_8                 3805 non-null   float32
 13  mfcc_9                 3805 non-null   float32
 14  mfcc_10                3805 non-null   float32
 15  mfcc

In [46]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5732 entries, 0 to 5731
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   age                    5732 non-null   int64  
 1   gender                 5732 non-null   object 
 2   respiratory_condition  5732 non-null   int64  
 3   fever_or_muscle_pain   5732 non-null   int64  
 4   mfcc_1                 5732 non-null   float32
 5   mfcc_2                 5732 non-null   float32
 6   mfcc_3                 5732 non-null   float32
 7   mfcc_4                 5732 non-null   float32
 8   mfcc_5                 5732 non-null   float32
 9   mfcc_6                 5732 non-null   float32
 10  mfcc_7                 5732 non-null   float32
 11  mfcc_8                 5732 non-null   float32
 12  mfcc_9                 5732 non-null   float32
 13  mfcc_10                5732 non-null   float32
 14  mfcc_11                5732 non-null   float32
 15  mfcc

In [47]:
# for i in df_full:
#     print(df_full[i].value_counts(), "\n")

In [48]:
tmp = []
for i in df_full["gender"]:
    if i == "male":
        tmp.append(1.0)
    elif i == "female":
        tmp.append(0.0)
    else:
        tmp.append(nan)
df_full["gender"] = tmp
df_full = df_full.dropna().reset_index(drop=True)

tmp = []
for i in df_test["gender"]:
    if i == "male":
        tmp.append(1.0)
    elif i == "female":
        tmp.append(0.0)
    else:
        tmp.append(0.5)
df_test["gender"] = tmp

In [49]:
df_full[num_vars + bin_vars] = df_full[num_vars + bin_vars].astype("float32")
df_test[num_vars + bin_vars] = df_test[num_vars + bin_vars].astype("float32")
df_full[target_var] = df_full[target_var].astype("int32")

# Feature Enginnering

In [50]:
# age discretization for 8, 4 groups

df_full["age_disc_8"] = pd.cut(df_full["age"], bins=[-np.inf, 10, 20, 30, 40, 50, 60, 70, np.inf], right=False, labels=list(range(8))).astype("float32")
df_full["age_disc_4"] = pd.cut(df_full["age"], bins=[-np.inf, 20, 40, 60, np.inf], right=False, labels=list(range(4))).astype("float32")

df_test["age_disc_8"] = pd.cut(df_test["age"], bins=[-np.inf, 10, 20, 30, 40, 50, 60, 70, np.inf], right=False, labels=list(range(8))).astype("float32")
df_test["age_disc_4"] = pd.cut(df_test["age"], bins=[-np.inf, 20, 40, 60, np.inf], right=False, labels=list(range(4))).astype("float32")

# cat_vars += ["age_disc_8", "age_disc_4"]

In [51]:
df_full = pd.concat([df_full.drop("age_disc_8", axis=1), dataframe(tf.keras.utils.to_categorical(df_full["age_disc_8"], num_classes=8, dtype='float32'), columns=["age_disc8_" + str(i) for i in range(8)])], axis=1)
df_full = pd.concat([df_full.drop("age_disc_4", axis=1), dataframe(tf.keras.utils.to_categorical(df_full["age_disc_4"], num_classes=4, dtype='float32'), columns=["age_disc4_" + str(i) for i in range(4)])], axis=1)

df_test = pd.concat([df_test.drop("age_disc_8", axis=1), dataframe(tf.keras.utils.to_categorical(df_test["age_disc_8"], num_classes=8, dtype='float32'), columns=["age_disc8_" + str(i) for i in range(8)])], axis=1)
df_test = pd.concat([df_test.drop("age_disc_4", axis=1), dataframe(tf.keras.utils.to_categorical(df_test["age_disc_4"], num_classes=4, dtype='float32'), columns=["age_disc4_" + str(i) for i in range(4)])], axis=1)

In [52]:
# covid19 symptom interaction (and, or operation)

df_full["symptom_and"] = (df_full["respiratory_condition"].astype("bool") & df_full["respiratory_condition"].astype("bool")).astype("float32")
df_full["symptom_or"] = (df_full["respiratory_condition"].astype("bool") | df_full["respiratory_condition"].astype("bool")).astype("float32")

df_test["symptom_and"] = (df_test["respiratory_condition"].astype("bool") & df_test["respiratory_condition"].astype("bool")).astype("float32")
df_test["symptom_or"] = (df_test["respiratory_condition"].astype("bool") | df_test["respiratory_condition"].astype("bool")).astype("float32")

bin_vars += ["symptom_and", "symptom_or"]

In [53]:
num_vars = diff(df_full.columns, bin_vars + cat_vars + mfcc_vars + [target_var])

In [54]:
num_vars

['age',
 'age_disc8_0',
 'age_disc8_1',
 'age_disc8_2',
 'age_disc8_3',
 'age_disc8_4',
 'age_disc8_5',
 'age_disc8_6',
 'age_disc8_7',
 'age_disc4_0',
 'age_disc4_1',
 'age_disc4_2',
 'age_disc4_3']

In [55]:
bin_vars

['gender',
 'respiratory_condition',
 'fever_or_muscle_pain',
 'symptom_and',
 'symptom_or']

In [56]:
cat_vars

[]

In [57]:
mfcc_vars

['mfcc_1',
 'mfcc_2',
 'mfcc_3',
 'mfcc_4',
 'mfcc_5',
 'mfcc_6',
 'mfcc_7',
 'mfcc_8',
 'mfcc_9',
 'mfcc_10',
 'mfcc_11',
 'mfcc_12',
 'mfcc_13',
 'mfcc_14',
 'mfcc_15',
 'mfcc_16',
 'mfcc_17',
 'mfcc_18',
 'mfcc_19',
 'mfcc_20',
 'mfcc_21',
 'mfcc_22',
 'mfcc_23',
 'mfcc_24',
 'mfcc_25',
 'mfcc_26',
 'mfcc_27',
 'mfcc_28',
 'mfcc_29',
 'mfcc_30',
 'mfcc_31',
 'mfcc_32']

In [58]:
df_full.head()

,age,gender,respiratory_condition,fever_or_muscle_pain,covid19,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32,age_disc8_0,age_disc8_1,age_disc8_2,age_disc8_3,age_disc8_4,age_disc8_5,age_disc8_6,age_disc8_7,age_disc4_0,age_disc4_1,age_disc4_2,age_disc4_3,symptom_and,symptom_or
0,24.0,0.0,0.0,1.0,0,-274.934753,29.345425,-19.152718,-7.836882,-9.094099,-8.553542,-32.653942,-12.341260,-5.151808,2.669801,-7.865372,10.570987,2.887322,4.517178,-8.187103,-0.459593,-6.641786,-0.670008,-4.505845,-1.137396,-0.307010,2.126430,-2.320942,2.150005,-0.925418,2.116030,-0.192730,2.417784,-4.736651,1.237788,-1.600425,-1.462419,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,51.0,1.0,0.0,0.0,0,-311.559875,52.482780,-0.094471,-11.066644,5.936104,-1.736337,-15.763054,-1.015405,-3.983850,7.575638,-10.625090,3.174337,-10.302692,1.403665,-10.509246,2.697304,-10.964950,-3.265628,0.137968,-0.563980,-2.603410,1.608950,-6.497139,0.543653,-6.263890,-2.386008,-6.744627,0.254181,-0.967560,-2.113267,-2.433431,0.881612,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,22.0,1.0,0.0,0.0,0,-438.286163,46.591045,-22.690268,-3.606580,-13.870162,0.272160,-9.013411,-9.623960,-7.755806,2.647471,-7.492092,1.029231,-1.479930,-3.566737,-3.868887,-2.445752,-2.173235,-2.239650,-5.330850,0.255554,-2.408603,-0.482843,-0.156195,-1.680288,2.617625,1.242552,-0.073267,-0.961353,-0.733599,-0.419574,0.796274,0.412893,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,29.0,0.0,1.0,0.0,0,-368.426025,46.939438,-7.443070,-3.694368,-20.511784,-9.271747,-10.894163,-11.857566,2.555381,-2.403064,-17.141569,4.961773,-8.069801,0.245035,-3.172873,-2.251958,-7.358308,-0.208508,-3.036400,-0.390360,-3.497282,-0.065154,-0.155832,3.839320,-2.503331,2.750769,1.758515,2.094565,0.295821,1.737586,-0.654200,1.847926,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,23.0,1.0,0.0,0.0,0,-535.192993,7.167374,-7.420830,2.231418,-5.301193,-0.646569,-6.103723,-0.876738,0.399676,0.912159,-5.177306,-1.339646,-3.864780,-1.080197,-2.543846,-1.409917,-3.337881,0.678286,-5.734587,2.078964,-2.041818,0.232061,-0.144892,-0.412889,-1.451348,0.237435,-0.971688,0.779252,-0.734346,1.141693,-0.498866,1.189332,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [59]:
df_test.head()

,age,gender,respiratory_condition,fever_or_muscle_pain,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32,age_disc8_0,age_disc8_1,age_disc8_2,age_disc8_3,age_disc8_4,age_disc8_5,age_disc8_6,age_disc8_7,age_disc4_0,age_disc4_1,age_disc4_2,age_disc4_3,symptom_and,symptom_or
0,48.0,0.0,1.0,0.0,-619.987915,9.328650,0.821095,2.373343,-0.171163,-1.189314,-3.113831,-0.829437,-2.221059,0.153571,-1.919320,0.480115,-1.152480,-0.310618,0.101920,-0.697523,-0.584930,-0.000477,-0.994546,0.114811,-0.488963,-0.081528,-0.313271,-0.053711,-0.029854,0.043562,-0.405293,-0.246975,-0.110936,0.019196,-0.155829,-0.130705,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1,24.0,0.0,0.0,0.0,-493.605713,8.492651,-2.758263,-2.801817,-2.770231,1.993531,-5.280821,-0.315033,0.839241,0.299542,-1.058011,3.300240,-2.794020,-0.276257,-0.596889,0.076233,-2.369861,0.703034,-1.309555,0.263056,-0.095919,-0.674825,-0.677968,-0.643835,-0.822677,-0.113844,-1.210391,0.273594,-0.685921,0.440101,-0.960277,0.371881,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,29.0,1.0,0.0,0.0,-1058.841797,-2.350609,-7.395449,-0.689041,-5.432968,1.596380,-8.221004,-1.885548,-5.361433,-1.276737,-4.865166,-3.347181,-9.040298,-2.554919,-2.569762,3.511593,-1.684626,-2.410390,-5.036191,-0.900126,-3.808790,4.415092,-0.015659,3.134083,2.587130,0.463058,-0.127755,1.612876,-3.393354,-2.874998,-1.302709,-2.915723,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,39.0,0.0,0.0,0.0,-362.161652,27.119738,-9.231675,9.747519,-1.013352,-4.020472,-13.982765,-11.758769,0.593285,-0.994719,-13.632190,0.128249,-3.824996,-5.212523,-5.973089,-0.987124,-7.543630,-5.531061,0.058560,-1.863635,1.387740,1.221440,-3.150051,-0.271802,0.034900,0.414227,-2.836006,0.105023,0.518484,2.139494,0.944032,1.029428,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,34.0,1.0,0.0,0.0,-387.592682,45.165874,-14.454832,-2.965205,-9.931789,10.073367,-0.863068,-6.342820,-6.891338,-9.549931,-6.007560,1.517536,-5.699743,3.236500,-4.142823,-7.910683,-6.125083,2.260159,-8.732250,3.884717,-5.244418,1.682219,-4.803989,-3.594219,0.289146,-5.298121,1.614137,3.108044,-5.373443,0.106868,2.054100,-1.994548,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [60]:
print(df_full.isna().sum().sum())
print(df_test.isna().sum().sum())

0
0


In [61]:
df_full = df_full.drop_duplicates().reset_index(drop=True)
df_full.sample(frac=1, random_state=42).reset_index(drop=True)

,age,gender,respiratory_condition,fever_or_muscle_pain,covid19,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,mfcc_22,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32,age_disc8_0,age_disc8_1,age_disc8_2,age_disc8_3,age_disc8_4,age_disc8_5,age_disc8_6,age_disc8_7,age_disc4_0,age_disc4_1,age_disc4_2,age_disc4_3,symptom_and,symptom_or
0,24.0,1.0,0.0,0.0,0,-548.837891,19.703192,1.444412,-8.035749,-6.256929,-0.908563,-6.069719,-1.307196,-6.660399,-2.900143,-3.823901,1.647556,-1.826432,-1.991143,0.221545,-0.846486,-2.018980,0.059515,-0.504477,1.046084,-1.128113,-0.435864,-0.676310,-0.522773,-0.925749,-0.323684,-1.905516,-0.241762,-0.506223,-0.533558,0.365484,0.588722,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,49.0,0.0,0.0,0.0,0,-409.823822,14.625152,-17.379200,4.448652,-0.241414,-1.601897,-21.544662,-3.597949,-13.139630,1.767516,-7.500328,-0.035637,-5.949492,6.866927,-8.541033,0.188570,-2.501330,-2.065399,-6.460501,2.110319,-7.322504,2.447815,-3.879176,1.942855,-2.196371,-0.291220,0.407009,0.400122,-3.908340,1.354989,-4.924136,1.645564,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,33.0,1.0,0.0,0.0,0,-530.992004,11.377920,0.333939,5.358803,2.507704,3.495510,-1.039859,-1.111229,-3.875501,-1.051320,-0.504201,-2.279216,-2.481425,-0.477329,-1.408024,-3.038084,-1.157562,-0.828378,-1.304849,-3.104608,0.188484,0.422442,-0.378243,1.287862,0.693949,0.821967,0.143595,-0.407756,0.473796,0.205788,-0.411068,0.591776,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,20.0,0.0,0.0,0.0,0,-453.309387,19.238050,-55.395695,12.609380,-26.218433,15.933744,-11.149816,-5.132318,-15.811329,3.013681,-12.704866,-8.174466,-27.457682,-9.946277,-15.640684,-5.811753,-6.374896,1.124129,-4.717995,7.041263,3.783126,4.701960,-1.955832,-2.208586,-9.483885,-4.026069,-1.548484,6.883171,5.334577,0.465221,-3.425792,-0.942889,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,46.0,1.0,0.0,0.0,0,-420.213776,7.490809,-12.221979,-0.381142,-8.284538,5.104037,-8.488156,0.117165,-2.996022,-1.714351,-5.504596,-5.059457,-2.210954,1.766647,-7.885905,-4.469393,-4.874714,-1.581117,-3.921628,5.206152,-0.717783,-3.284468,-3.955352,1.262931,-1.704155,0.301872,-0.571943,2.580835,-1.371098,1.966161,-0.215612,0.906044,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3774,12.0,1.0,0.0,0.0,0,-934.885132,-7.388165,-2.893190,4.437783,-8.830144,0.967720,-14.739068,-6.797158,0.182026,7.523750,-10.370166,-0.983729,-4.535510,4.329366,-10.930986,4.954981,-6.925780,-0.891341,-11.579190,-4.912291,-11.541049,4.959506,-3.884153,-1.859265,-4.309305,3.094619,-8.801568,-4.925787,-6.118340,-1.690299,-3.229438,1.839331,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3775,40.0,1.0,0.0,0.0,0,-517.425720,21.314951,-5.152871,6.703038,-5.431258,-2.128231,-6.593565,-3.367700,-5.893108,-1.364386,-4.397043,-0.188966,-4.695415,-0.442561,-1.138519,-2.001806,-1.824370,-0.122556,-3.358488,0.531409,-1.451319,-1.154755,-0.707734,-0.119056,-1.428291,0.820588,-1.515648,-0.113138,-0.183102,0.466720,-0.824456,0.093877,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3776,52.0,1.0,0.0,0.0,0,-363.085632,30.844067,-25.609774,9.367500,-15.117414,5.021363,-9.654868,-8.820436,-8.844234,-5.219638,-5.832464,-7.009341,-2.926299,-5.335454,-7.534382,-4.063085,-2.567483,-3.355904,-2.272799,1.533910,-3.850093,0.998559,0.545355,1.393837,2.062032,0.874791,0.073296,3.404635,0.529662,0.854970,-0.077099,0.765816,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3777,37.0,0.0,0.0,0.0,0,-562.892578,26.073837,-12.282734,4.154382,-15.053585,-1.696431,-12.699330,-3.277766,-4.067583,0.936061,-6.888280,0.501659,-3.568062,-5.716290,-3.088

# Define utility funcions

In [73]:
# optuna function
def optuna_objective_function(trial: Trial, fold, train_x, train_y, train_groups, val_x, val_y, val_groups, categoIdx,
                              model_name, output_container, ntrees=1000, eta=1e-2):
    if model_name == "LGB_GOSS":
        tuning_params = {
            "num_leaves": trial.suggest_categorical("num_leaves", [pow(2, i) - 1 for i in [4, 5, 6, 7, 8]]),
            "subsample": trial.suggest_float("subsample", 0.5, 0.8, step=0.05),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.8, step=0.05),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.001, 10.0, log=True),
            "min_child_weight": trial.suggest_float("min_child_weight", 0.001, 10.0, log=True),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 99, log=True),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.2, 5.0, log=True),
        }
        cb_list = [
            lgb.early_stopping(stopping_rounds=int(ntrees * 0.2), first_metric_only=True, verbose=False, min_delta=0.001),
        ]

        model = lgb.LGBMClassifier(boosting_type="goss", objective="binary",
                                   n_estimators=ntrees, learning_rate=eta,
                                   random_state=fold, device_type="gpu",
                                   verbose=-1, **tuning_params)
        model.fit(train_x, train_y, categorical_feature=categoIdx,
                  eval_set=(val_x,val_y), eval_metric="auc", callbacks=cb_list)
    elif model_name == "XGB_GBT":
        tuning_params = {
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "subsample": trial.suggest_float("subsample", 0.5, 0.8, step=0.05),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.8, step=0.05),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 10.0, step=0.01),
            "min_child_weight": trial.suggest_float("min_child_weight", 0.01, 10.0, step=0.01),
            "gamma": trial.suggest_float("gamma", 0.01, 10.0, step=0.01),
            "max_delta_step":  trial.suggest_float("max_delta_step", 0.01, 10.0, step=0.01)
        }
        model = xgb.XGBRanker(booster="gbtree", objective="rank:ndcg",
                            tree_method="gpu_hist", sampling_method="gradient_based",
                            n_estimators=int(ntrees * 0.2),
                            learning_rate=eta / 10,
                            random_state=fold, verbosity=0,
                            **tuning_params)
        model.fit(train_x, train_y, group=train_groups, verbose=False)
    elif model_name == "CAT_GBM":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.05, 0.95, step=0.05),
            # "rsm": trial.suggest_float("rsm", 0.5, 0.9, step=0.05),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.01, 10.0, step=0.01),
            "random_strength": trial.suggest_float("random_strength", 0.01, 1.5, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 99, step=2),  
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }

        model = cat.CatBoostClassifier(boosting_type="Plain", loss_function="MultiClass", task_type="GPU",
                                    n_estimators=int(ntrees * 0.2),
                                    learning_rate=eta / 10,
                                    one_hot_max_size=3, leaf_estimation_method="Gradient",
                                    # leaf_estimation_iterations=5,
                                    # max_ctr_complexity=2,
                                    logging_level="Silent", random_state=fold, thread_count=cpu_count(),
                                    **tuning_params)
        model.fit(train_x, train_y, cat_features=categoIdx)
    elif model_name == "CAT_ORD":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.05, 0.95, step=0.05),
            # "rsm": trial.suggest_float("rsm", 0.5, 0.9, step=0.05),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.01, 10.0, step=0.01),
            "random_strength": trial.suggest_float("random_strength", 0.01, 1.5, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 99, step=2),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }
        
        model = cat.CatBoostClassifier(boosting_type="Ordered", loss_function="MultiClass", task_type="GPU",
                                    n_estimators=int(ntrees * 0.2),
                                    learning_rate=eta / 10,
                                    one_hot_max_size=3, leaf_estimation_method="Gradient",
                                    # leaf_estimation_iterations=5,
                                    # max_ctr_complexity=2,
                                    logging_level="Silent", random_state=fold, thread_count=cpu_count(),
                                    **tuning_params)
        model.fit(train_x, train_y, cat_features=categoIdx)
    else:
        print("unknown")
        return -1
    
    pred = model.predict_proba(val_x)
    # f1 score
    optuna_score = metrics.f1_score(val_y.tolist(), [1.0 if i > threshold else 0.0 for i in pred[:,1]])

    if optuna_score > output_container["score"]:
        output_container["model"] = model
        output_container["pred"] = pred
        output_container["score"] = optuna_score

    return optuna_score

# Training

In [74]:
architecture_name = "lgb_goss_thresholdOri_tuningPosWeight_try1"
createFolder(folder_path + "architecture/" + architecture_name)

In [75]:
def do_fold_training(fold, train_idx, val_idx, sample_weight=None, finetuning=False):
    tmp_time = time()
    print("\n===== Fold", fold, "=====\n")
    global val_pred, test_pred

    wandb.init(
        project="dacon_covid19_classification",
        group=architecture_name,
        name="fold_" + str(fold)
    ); wandb.config.step = 0

    train_x = df_full.iloc[train_idx][num_vars + bin_vars + mfcc_vars].copy()
    train_y = df_full.iloc[train_idx][target_var].copy()
    
    val_x = df_full.iloc[val_idx][num_vars + bin_vars + mfcc_vars].copy()
    val_y = df_full.iloc[val_idx][target_var].copy()
    
    output_container = {"model": None, "pred": None, "score": -np.inf}
    optuna_timout = int(6 * 3600 / kfolds_spliter.get_n_splits())
    optuna_study = create_study(direction='maximize', sampler=TPESampler())
    optuna_study.optimize(
        lambda trial: optuna_objective_function(
            trial, fold, train_x, train_y, None, val_x, val_y, None, categoIdx=None, model_name="LGB_GOSS", output_container=output_container, ntrees=ntrees, eta=eta
        ),
        n_jobs=1, n_trials=300, timeout=optuna_timout
    )
    
    model_list.append(output_container["model"])
    params_list.append(optuna_study.best_params)
    print("fold", fold, "best params :", params_list[-1])
    val_pred[val_idx] = output_container["pred"]
    fold_metric.append(output_container["score"])
    print("fold", fold, "score :", fold_metric[-1])

    test_x = df_test[num_vars + bin_vars + mfcc_vars].copy()
    test_pred += model_list[-1].predict_proba(test_x) / n_folds

    wandb.log({"fold": fold,
               "logloss": metrics.log_loss(val_y.tolist(), val_pred[val_idx, 1]),
               "auc": metrics.roc_auc_score(val_y.tolist(), val_pred[val_idx, 1]),
               "f1_score": fold_metric[-1]})
    wandb.finish()
 
    print("fold", fold, "time to training :", round(time() - tmp_time, 3))

In [76]:
# learning parameter setting
# The range of converged trees is about 2000 ~ 4000
ntrees = 5000
eta = 5e-3

# ntrees = 1000
# eta = 1e-2

threshold = round(df_full[target_var].mean(), 5)
# threshold = 0.5

In [77]:
model_list = []
params_list = []
fold_metric = []

val_pred = np.zeros(shape=(df_full.shape[0], 2))
test_pred = np.zeros(shape=(df_test.shape[0], 2))
seed_everything()

n_folds = 5
kfolds_spliter = StratifiedKFold(n_folds, shuffle=True, random_state=42)

start_time_training = time()
# fold training
for fold, (train_idx, val_idx) in enumerate(kfolds_spliter.split(df_full, y=df_full[target_var])):
    start_mem = memory_usage()   
    do_fold_training(fold, train_idx, val_idx, None, finetuning=False)
    gc.collect()
    end_mem = memory_usage()
    print("@Memory leaked :", end_mem - start_mem, "\n")
end_time_training = time()

[debug] memory usage:   1277.859 MB

===== Fold 0 =====



[I 2022-06-15 08:04:54,376] A new study created in memory with name: no-name-e7eb8dee-1e31-433f-a6f9-d6f4f02f1983
[I 2022-06-15 08:05:07,206] Trial 0 finished with value: 0.19047619047619047 and parameters: {'num_leaves': 15, 'subsample': 0.55, 'colsample_bytree': 0.75, 'reg_lambda': 0.004662623012722093, 'min_child_weight': 0.05219057226452718, 'min_child_samples': 7, 'scale_pos_weight': 2.678729983985663}. Best is trial 0 with value: 0.19047619047619047.
[I 2022-06-15 08:05:37,622] Trial 1 finished with value: 0.10526315789473684 and parameters: {'num_leaves': 127, 'subsample': 0.75, 'colsample_bytree': 0.65, 'reg_lambda': 0.004946684076724309, 'min_child_weight': 0.028529071283832988, 'min_child_samples': 12, 'scale_pos_weight': 0.22191604167309462}. Best is trial 0 with value: 0.19047619047619047.
[I 2022-06-15 08:05:48,459] Trial 2 finished with value: 0.2096774193548387 and parameters: {'num_leaves': 255, 'subsample': 0.6, 'colsample_bytree': 0.55, 'reg_lambda': 0.031744118983758

fold 0 best params : {'num_leaves': 127, 'subsample': 0.7, 'colsample_bytree': 0.65, 'reg_lambda': 0.012891205412474448, 'min_child_weight': 7.705042383396356, 'min_child_samples': 56, 'scale_pos_weight': 0.3331855342530665}
fold 0 score : 0.30666666666666664
fold 0 time to training : 2662.702
[debug] memory usage:   1611.039 MB
@Memory leaked : 333.1796875 

[debug] memory usage:   1611.039 MB

===== Fold 1 =====



auc,▁
f1_score,▁
fold,▁
logloss,▁
auc,0.70279
f1_score,0.30667
fold,0
logloss,0.27362


[I 2022-06-15 08:49:17,647] A new study created in memory with name: no-name-43c22763-e800-4708-b08b-c121213804ce
[I 2022-06-15 08:49:28,939] Trial 0 finished with value: 0.13953488372093023 and parameters: {'num_leaves': 63, 'subsample': 0.75, 'colsample_bytree': 0.7, 'reg_lambda': 0.003483595071880817, 'min_child_weight': 1.1672742875825375, 'min_child_samples': 8, 'scale_pos_weight': 0.21775779383831642}. Best is trial 0 with value: 0.13953488372093023.
[I 2022-06-15 08:49:35,386] Trial 1 finished with value: 0.2037037037037037 and parameters: {'num_leaves': 15, 'subsample': 0.8, 'colsample_bytree': 0.75, 'reg_lambda': 0.0038154107989567714, 'min_child_weight': 0.03411186908429238, 'min_child_samples': 64, 'scale_pos_weight': 0.44045639882580134}. Best is trial 1 with value: 0.2037037037037037.
[I 2022-06-15 08:49:41,836] Trial 2 finished with value: 0.16991150442477873 and parameters: {'num_leaves': 255, 'subsample': 0.6, 'colsample_bytree': 0.7, 'reg_lambda': 0.05652943031359826, 

...

[I 2022-06-15 10:18:05,283] A new study created in memory with name: no-name-ac541e7c-dfa1-4988-85a2-f1b0067d430c
[I 2022-06-15 10:18:13,993] Trial 0 finished with value: 0.14758269720101783 and parameters: {'num_leaves': 15, 'subsample': 0.8, 'colsample_bytree': 0.75, 'reg_lambda': 0.06598719437070857, 'min_child_weight': 0.07341795897534471, 'min_child_samples': 13, 'scale_pos_weight': 3.833361320185802}. Best is trial 0 with value: 0.14758269720101783.
[I 2022-06-15 10:18:23,821] Trial 1 finished with value: 0.34234234234234234 and parameters: {'num_leaves': 15, 'subsample': 0.55, 'colsample_bytree': 0.6, 'reg_lambda': 0.029460175499604953, 'min_child_weight': 4.911825355110786, 'min_child_samples': 45, 'scale_pos_weight': 0.3397762812490352}. Best is trial 1 with value: 0.34234234234234234.
[I 2022-06-15 10:18:37,290] Trial 2 finished with value: 0.18317757009345795 and parameters: {'num_leaves': 31, 'subsample': 0.75, 'colsample_bytree': 0.7, 'reg_lambda': 0.12367469364025895, 'mi

fold 3 best params : {'num_leaves': 31, 'subsample': 0.55, 'colsample_bytree': 0.7, 'reg_lambda': 0.10646264677147098, 'min_child_weight': 0.06695133374374905, 'min_child_samples': 54, 'scale_pos_weight': 0.46723405257847567}
fold 3 score : 0.37142857142857144
fold 3 time to training : 2994.486
[debug] memory usage:   2850.066 MB
@Memory leaked : 531.2109375 

[debug] memory usage:   2850.066 MB

===== Fold 4 =====



auc,▁
f1_score,▁
fold,▁
logloss,▁
auc,0.69619
f1_score,0.37143
fold,3
logloss,0.26193


[I 2022-06-15 11:08:00,415] A new study created in memory with name: no-name-6f5afc9e-acfe-446a-b3fb-fa871b0e92e1
[I 2022-06-15 11:08:07,883] Trial 0 finished with value: 0.25287356321839083 and parameters: {'num_leaves': 31, 'subsample': 0.6, 'colsample_bytree': 0.8, 'reg_lambda': 0.0018637949545601022, 'min_child_weight': 0.0019382248219826344, 'min_child_samples': 11, 'scale_pos_weight': 0.44342614458808155}. Best is trial 0 with value: 0.25287356321839083.
[I 2022-06-15 11:08:15,126] Trial 1 finished with value: 0.2637362637362638 and parameters: {'num_leaves': 31, 'subsample': 0.6, 'colsample_bytree': 0.8, 'reg_lambda': 0.10428526370531109, 'min_child_weight': 0.04877141481581596, 'min_child_samples': 17, 'scale_pos_weight': 0.41564876060175965}. Best is trial 1 with value: 0.2637362637362638.
[I 2022-06-15 11:08:20,893] Trial 2 finished with value: 0.14496314496314497 and parameters: {'num_leaves': 15, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_lambda': 0.0015971050977337488

fold 4 best params : {'num_leaves': 255, 'subsample': 0.55, 'colsample_bytree': 0.5, 'reg_lambda': 0.013756562318542527, 'min_child_weight': 7.1411139090204365, 'min_child_samples': 7, 'scale_pos_weight': 0.4603738092431843}
fold 4 score : 0.34355828220858897
fold 4 time to training : 1879.072
[debug] memory usage:   3151.613 MB
@Memory leaked : 301.546875 



In [88]:
val_pred[:50]

array([[0.87925665, 0.12074335],
       [0.92666092, 0.07333908],
       [0.9285571 , 0.0714429 ],
       [0.9410917 , 0.0589083 ],
       [0.96455732, 0.03544268],
       [0.92235404, 0.07764596],
       [0.96764886, 0.03235114],
       [0.92173617, 0.07826383],
       [0.96516211, 0.03483789],
       [0.98070449, 0.01929551],
       [0.92205456, 0.07794544],
       [0.92793452, 0.07206548],
       [0.96286943, 0.03713057],
       [0.92209745, 0.07790255],
       [0.9284822 , 0.0715178 ],
       [0.92196042, 0.07803958],
       [0.97908977, 0.02091023],
       [0.92203793, 0.07796207],
       [0.96592871, 0.03407129],
       [0.96763714, 0.03236286],
       [0.92203942, 0.07796058],
       [0.92694007, 0.07305993],
       [0.9551733 , 0.0448267 ],
       [0.96635216, 0.03364784],
       [0.94538578, 0.05461422],
       [0.92155003, 0.07844997],
       [0.92310567, 0.07689433],
       [0.92201498, 0.07798502],
       [0.98393326, 0.01606674],
       [0.92218149, 0.07781851],
       [0.

In [89]:
avg_score = 0
for idx, value in enumerate(fold_metric):
    print("fold", idx, "score :", value)
    avg_score += value / n_folds
print("average score :", avg_score)

fold 0 score : 0.30666666666666664
fold 1 score : 0.3539823008849557
fold 2 score : 0.3232323232323232
fold 3 score : 0.37142857142857144
fold 4 score : 0.34355828220858897
average score : 0.3397736288842212


# Submission

In [79]:
dataframe(val_pred).to_csv(folder_path + "architecture/" + architecture_name + "/valPred_" + architecture_name + ".csv", index=False)
dataframe(test_pred).to_csv(folder_path + "architecture/" + architecture_name + "/testPred_" + architecture_name + ".csv", index=False)

In [80]:
submission = pd.read_csv(folder_path + 'open.zip (Unzipped Files)/sample_submission.csv')
submission['covid19'] = [1 if i > threshold else 0 for i in test_pred[:,1]]
submission.to_csv(folder_path + "architecture/" + architecture_name + "/submission_" + architecture_name + ".csv", index=False)